Muntele Olimp

# Clasa Nod 

In [1]:
class Nod:
  def __init__(self, informatie, parinte = None, g=0, h=0):
    ''' Constructorul clasei Nod.

    :param informatie: informatia nodului curent
    :param parinte: pointer catre parintele nodului curent
    :param g: lungimea drumului de la nodul start pana la nodul curent
    :param h: estimarea distantei de la nodul curent pana la cel mai apropiat nod scop
    '''
    self.informatie = informatie
    self.parinte = parinte
    self.g = g
    self.h = h
    self.f = g + h

  def __repr__(self):
    return str(self.informatie)

  def __str__(self):
    return repr(self) + ' (' + ' -> '.join([repr(x) for x in self.drumRadacina()]) + ')'

  def __eq__(self, cls):
     return self.informatie == cls.informatie

  def __lt__(self, cls):
    ''' Defineste relatie < intre doua instante ale clasei Nod. '''
    return self.f < cls.f or (self.f == cls.f and self.g > cls.g)

  def drumRadacina(self):
    ''' Calculeaza lista nodurilor de la radacina pana la nodul curent. '''
    if self.parinte is None:
      return [self]
    return self.parinte.drumRadacina() + [self]

  def vizitat(self):
    ''' Returneaza True daca nodul a fost deja vizitat, False altfel. '''
    return len([1 for nod in self.drumRadacina() if nod == self]) > 1

# Clasa Graf

In [2]:
class Graf:
  def __init__(self, nodStart, noduriScop, muchii, estimari):
    ''' Constructorul clasei Graf.

    :param nodStart: informatia din nodul start
    :param noduriScop: lista de informatii din nodurile scop
    :param muchii: lista de muchii de forma (informatieNod1, informatieNod2, cost)
    :param estimari: dictionar de estimari pentru noduri: (nod: estimare)
    '''
    self.nodStart = Nod(nodStart)
    self.noduriScop = noduriScop
    self.muchii = muchii
    self.estimari = estimari

  def scop(self, nod):
    return nod.informatie in self.noduriScop

  def estimeaza_h(self, informatie_nod):
    return self.estimari[informatie_nod]

  def succesori(self, nod):
    ''' Primeste un nod al arborelui de parcurgere si returnează lista succesorilor directi
      ai nodului care nu au fost vizitati pe ramura curenta.

    :param nod: un nod oarecare
    :return: lista succesorilor nodului
    '''
    succesori = []

    for nodNou in self.muchii[nod.informatie]:

      nodCurent = Nod(nodNou[0], nod, nod.g + nodNou[1], self.estimeaza_h(nodNou[0]))
      if nodCurent.vizitat():
        continue

      succesori.append((nodCurent, nodNou[1]))

    return succesori

# Implementare euristica, A* si IDA* 

In [7]:
import heapq
from collections import deque

def calculeaza_euristica(muchii, noduri_scop):
    h = {nod: float('inf') for nod in muchii}
    coada = deque()

    for scop in noduri_scop:
        h[scop] = 0
        coada.append(scop)

    while coada:
        nod = coada.popleft()
        for vecin in muchii[nod]:
            if h[vecin[0]] > h[nod] + vecin[1]:
                h[vecin[0]] = h[nod] + vecin[1]
                coada.append(vecin[0])
    return h

def a_star(graf, nrPasi):
    open = [graf.nodStart]
    closed = {}

    pasi_efectuati = 0

    while open and pasi_efectuati < nrPasi:

        nodCurent = heapq.heappop(open)
        closed[nodCurent.informatie] = nodCurent

        succesori = graf.succesori(nodCurent)

        for succesor in succesori:
            nodVechiOpen = next((nod for nod in open if nod.informatie == succesor[0].informatie), None)
            nodVechiClose = closed.get(succesor[0].informatie)

            nodNou = None

            if nodVechiOpen:
                if succesor[0] < nodVechiOpen:
                    open.remove(nodVechiOpen)
                    heapq.heapify(open)
                    nodNou = succesor[0]

            elif nodVechiClose:
                if succesor[0] < nodVechiClose:
                    closed.pop(nodVechiClose.informatie)
                    nodNou = succesor[0]

            else:
                nodNou = succesor[0]

            if nodNou:
                if graf.scop(nodNou):
                    return [nodNou.informatie]
                heapq.heappush(open, nodNou)

        pasi_efectuati += 1

    return open


In [ ]:
# Afla drumul optim de la nodul de start la un nod scop, in loc de lista close
def ida_star(graf, nrPasi):
    def cautare_limitata(nodCurent, limita, pasi_ramasi):
        if graf.scop(nodCurent):
            return [nodCurent.informatie], True, nodCurent.f
            
        if pasi_ramasi <= 0 or nodCurent.f > limita:
            return None, False, nodCurent.f
        
        limita_minima = float('inf')
        succesori = graf.succesori(nodCurent)
        
        if not succesori:
            return None, False, nodCurent.f
            
        for succesor in succesori:
            nod = succesor[0]
            rezultat, gasit, limita_noua = cautare_limitata(nod, limita, pasi_ramasi - 1)
            
            if gasit:
                return [nodCurent.informatie] + rezultat, True, limita_noua
                
            limita_minima = min(limita_minima, limita_noua)
            
        return None, False, limita_minima

    nodStart = graf.nodStart
    if graf.scop(nodStart):
        return [nodStart.informatie]
        
    limita = nodStart.f
    
    for i in range(nrPasi):
        rezultat, gasit, limita_noua = cautare_limitata(nodStart, limita, nrPasi)
        
        if gasit:
            return rezultat
            
        if limita_noua == float('inf'):
            return "Nu s-a gasit un nod scop"
            
        limita = limita_noua
        
    return "Nu s-a gasit un nod scop"

Muchii si functia de rezolvare

In [ ]:
# nod: [(nod, cost), ...]
MUCHII = {
    1: [(2, 3), (10, 3)],
    2: [(1, 3), (3, 3), (5, 1)],
    3: [(2, 3), (15, 3)],
    4: [(5, 2), (11, 2)],
    5: [(2, 1), (4, 2), (6, 2), (8, 1)],
    6: [(5, 2), (14, 2)],
    7: [(8, 1), (12, 1)],
    8: [(5, 1), (7, 1), (9, 1)],
    9: [(8, 1), (13,1)],
    10: [(1, 3), (11, 1), (22, 3)],
    11: [(4, 2), (10, 1), (12, 1), (19, 2)],
    12: [(7, 1), (11, 1), (16, 1)],
    13: [(9, 1), (14, 1), (18, 1)],
    14: [(6, 2), (13, 1), (15, 1), (21, 2)],
    15: [(3, 3), (14, 1), (24, 3)],
    16: [(12, 1), (17, 1)],
    17: [(16, 1), (18, 1), (20, 1)],
    18: [(13, 1), (17, 1)],
    19: [(11, 2), (20, 2)],
    20: [(17, 1), (19, 2), (21, 2), (23, 1)],
    21: [(20, 2), (14, 2)],
    22: [(10, 3), (23, 3)],
    23: [(20, 1), (22, 3), (24, 3)],
    24: [(15, 3), (23, 3)]
}

def rezolvare(nod_start, noduri_scop, nr_pasi, algoritm):

    estimari = calculeaza_euristica(MUCHII, noduri_scop)
    graf = Graf(nod_start, noduri_scop, MUCHII, estimari)

    if algoritm == "A*":
        open = a_star(graf, nr_pasi)
        if len(open) == 1:
            print(open[0])
        else:
            rez = []
            for nod in open:
                rez.append(f"{nod.informatie} ({nod.f})")
            print(rez)
    elif algoritm == "IDA*":
        rezultat = ida_star(graf, nr_pasi)
        print(rezultat)
    else:
        print("Algoritm gresit")



In [34]:
NOD_START = 12
NODURI_SCOP = [1, 20]
NR_PASI = 1
ALGORITM = "A*"

rezolvare(NOD_START, NODURI_SCOP, NR_PASI, ALGORITM)
rezolvare(NOD_START, NODURI_SCOP, 2, "IDA*")
rezolvare(NOD_START, NODURI_SCOP, 3, "IDA*")


['16 (3)', '11 (5)', '7 (5)']
Nu s-a gasit un nod scop
[12, 16, 17, 20]


# Documentatie #

# Justificare euristici #
Am folosit o functie euristica bazata pe distanta intre nodurile scop si restul nodurilor din graf, practic distanta minima de la oricare nod pana la un nod scop. Aceasta euristica este eficienta, deoarece este admisibila si reprezinta distanta efectiva in graf intre noduri. Ca si implementare, folosesc un dictionar cu fiecare nod si o parcurgere BFS din nodurile scop, actualizand aferent dictionarul in functie de ce noduri parcurg.

# Justificare ponderi #
Ponderile intre noduri sunt 1, 2, respectiv 3, in functie de distanta dintre ele. Am ales aceste dimensiuni cu gandul ca intre patrate si in patratul cel mic ponderile sunt de o unitate, in patratul mijlociu sunt de 2 unitati, iar in patratul cel mare din exterior sunt de 3 unitati. Se respecta proprietatea din cerinta: cost(1, 2) > cost(4, 5) > cost(7, 8) = 3 > 2 > 1.

# Despre algoritmi si implementari #
Algoritmul A* afiseaza lista open dupa numarul de pasi dorit, iar algoritmul IDA*, in lipsa listei close din implementare, afiseaza drumul pana la un nod scop daca acesta a fost gasit, sau mesajul "Nu s-a gasit un nod scop" in cazul in care dupa numarul de pasi nu s-a gasit un nod scop.